In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf
import pathlib
import matplotlib.pyplot as plt
import datetime

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv3D, Flatten, Dropout, MaxPooling3D, GlobalMaxPooling3D

from generators import VideoDataGenerator
from loader import get_classes

AUTOTUNE = tf.data.experimental.AUTOTUNE

print("Tensorflow Version: ", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
data_dir = pathlib.Path("./data")
classes = get_classes(data_dir / "classes.txt")

params = {
    'color_mode': 'landmarks',
    'optical_flow': False,
    'batch_size': 4,
    'shuffle': True,
    'classes': classes
}

partition = {
    'train': list((data_dir / "Train").glob("**/*.mp4")),
    'test': list((data_dir / "Test").glob("**/*.mp4"))
}

training_generator = VideoDataGenerator(partition['train'], **params)
validation_generator = VideoDataGenerator(partition['test'], **params)

In [ ]:
#model = Sequential([
#    Conv3D(16, (3, 7, 7), activation='relu', input_shape=(15, 240, 320, training_generator.n_channels)),
#    Conv3D(24, (3, 7, 7), activation='relu'),
#    MaxPooling3D((1, 3, 3)),
#    Conv3D(32, (3, 5, 7), activation='relu'),
#    Conv3D(64, (3, 5, 7), activation='relu'),
#    MaxPooling3D((1, 2, 2)),
#    Conv3D(128, (3, 3, 5), activation='relu'),
#    Conv3D(256, (3, 3, 5), activation='relu'),
#    Conv3D(512, (3, 3, 5), activation='relu'),
#    MaxPooling3D((1, 2, 2)),
#    Flatten(),
#    Dense(512, activation='relu'),
#    Dense(256, activation='relu'),
#    Dense(2, activation='softmax')
#])

model = Sequential([
    Conv3D(16, (1, 4, 3), activation='relu', input_shape=(training_generator.dim)),
    MaxPooling3D(),
    Conv3D(32, (1, 4, 3), activation='relu'),
    MaxPooling3D(),
    Conv3D(64, (1, 4, 3), activation='relu'),
    MaxPooling3D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(training_generator, validation_data=validation_generator, epochs=100, callbacks=[callback])

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
export_time = datetime.datetime.now()
export_path = pathlib.Path("export/model_{}-{}-{}-{}{}.h5".format(export_time.year, export_time.month, export_time.day, export_time.hour, export_time.minute))
print("Exporting trained model to {}".format(export_path.absolute()))

model.save(export_path)